# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=True, enhanced=False)


Processing dataset:   0%|          | 0/2 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 2/2 [00:19<00:00,  9.87s/it]


# Methods

# Main

In [5]:
data_loader.processed_incidents_df.dtypes


triage_impression_during_call            object
resource_id                              object
resource_type                            object
time_call_received               datetime64[ns]
time_call_processed              datetime64[ns]
time_ambulance_notified          datetime64[ns]
time_dispatch                    datetime64[ns]
time_arrival_scene               datetime64[ns]
time_departure_scene             datetime64[ns]
time_arrival_hospital            datetime64[ns]
time_available                   datetime64[ns]
grid_id                                   int64
x                                         int64
y                                         int64
x_accurate                                int64
y_accurate                                int64
longitude                               float64
latitude                                float64
region                                   object
urban_settlement                           bool
dtype: object

In [ ]:
# Assuming df is your DataFrame
grouped = data_loader.processed_incidents_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])
duplicate_counts = grouped.size()

# Count the total number of rows where time_call_received, x_accurate, and y_accurate are the same
total_duplicates = duplicate_counts[duplicate_counts > 1].sum()

total_duplicates


In [ ]:
# Assuming df is your DataFrame
grouped = data_loader.processed_incidents_df.groupby(['x_accurate', 'y_accurate'])
duplicate_counts = grouped.size()

# Count the total number of rows where time_call_received, x_accurate, and y_accurate are the same
total_duplicates = duplicate_counts[duplicate_counts > 1].sum()

total_duplicates

In [ ]:
# Group by time_call_received, x_accurate, and y_accurate
grouped = data_loader.processed_incidents_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])
duplicate_counts = grouped.size().reset_index(name='counts')

# Filter to get only duplicates
duplicates = duplicate_counts[duplicate_counts['counts'] > 1].copy()

# Extract the year from time_call_received for sorting
duplicates['year'] = duplicates['time_call_received'].dt.year

# Sort the duplicates by year
sorted_duplicates = duplicates.sort_values(by='year')

# Count the occurrences of each year in the sorted duplicates
year_occurrences = sorted_duplicates['year'].value_counts().sort_index()

year_occurrences

In [7]:
# Extract the year from time_call_received
data_loader.processed_incidents_df['year'] = data_loader.processed_incidents_df['time_call_received'].dt.year

# Keep only the rows where the year is between 2016 and 2018
filtered_df = data_loader.processed_incidents_df[(data_loader.processed_incidents_df['year'] >= 2016) & (data_loader.processed_incidents_df['year'] <= 2018)]

# Drop the 'year' column if it's no longer needed
filtered_df = filtered_df.drop(columns=['year'])

filtered_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 580737 entries, 5199 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  580728 non-null  object        
 1   resource_id                    580737 non-null  object        
 2   resource_type                  580737 non-null  object        
 3   time_call_received             580737 non-null  datetime64[ns]
 4   time_call_processed            580737 non-null  datetime64[ns]
 5   time_ambulance_notified        580419 non-null  datetime64[ns]
 6   time_dispatch                  563737 non-null  datetime64[ns]
 7   time_arrival_scene             513997 non-null  datetime64[ns]
 8   time_departure_scene           432640 non-null  datetime64[ns]
 9   time_arrival_hospital          430456 non-null  datetime64[ns]
 10  time_available                 580714 non-null  datetime64[ns]
 11  gr

In [ ]:
# Group by time_call_received, x_accurate, and y_accurate
grouped = filtered_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])
duplicate_counts = grouped.size().reset_index(name='counts')

# Filter to get only duplicates
duplicates = duplicate_counts[duplicate_counts['counts'] > 1].copy()

# Extract the year from time_call_received for sorting
duplicates['year'] = duplicates['time_call_received'].dt.year

# Sort the duplicates by year
sorted_duplicates = duplicates.sort_values(by='year')

# Count the occurrences of each year in the sorted duplicates
year_occurrences = sorted_duplicates['year'].value_counts().sort_index()

year_occurrences


In [ ]:
# Group by day and count incidents
daily_incidents = data_loader.processed_incidents_df.groupby(data_loader.processed_incidents_df['time_call_received'].dt.date).size()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(daily_incidents.index, daily_incidents.values)

# Formatting x-axis to show only the year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Total Incidents')
plt.title('Total Incidents Per Day Over Years')

plt.grid(True)
plt.show()


In [ ]:
# Group by day and count incidents
daily_incidents = filtered_df.groupby(filtered_df['time_call_received'].dt.date).size()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(daily_incidents.index, daily_incidents.values)

# Formatting x-axis to show only the year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Total Incidents')
plt.title('Total Incidents Per Day Over Years')

plt.grid(True)
plt.show()

In [ ]:
filtered_df = filtered_df.dropna(subset=["region"])


In [ ]:
# Group by day and count incidents
daily_incidents = filtered_df.groupby(filtered_df['time_call_received'].dt.date).size()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(daily_incidents.index, daily_incidents.values)

# Formatting x-axis to show only the year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Total Incidents')
plt.title('Total Incidents Per Day Over Years')

plt.grid(True)
plt.show()

In [ ]:
# Group by time_call_received, x_accurate, and y_accurate
grouped = filtered_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])
duplicate_counts = grouped.size().reset_index(name='counts')

# Filter to get only duplicates
duplicates = duplicate_counts[duplicate_counts['counts'] > 1].copy()

# Extract the year from time_call_received for sorting
duplicates['year'] = duplicates['time_call_received'].dt.year

# Sort the duplicates by year
sorted_duplicates = duplicates.sort_values(by='year')

# Count the occurrences of each year in the sorted duplicates
year_occurrences = sorted_duplicates['year'].value_counts().sort_index()

year_occurrences

In [ ]:
# Group by time_call_received, x_accurate, and y_accurate
grouped = filtered_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])

# Identifying duplicate groups (keeping one per group)
duplicates = grouped.filter(lambda x: len(x) > 1).drop_duplicates(subset=['time_call_received', 'x_accurate', 'y_accurate'])

# Extract the year from time_call_received
duplicates['year'] = duplicates['time_call_received'].dt.year

# Count the occurrences of each year in the duplicates
year_occurrences = duplicates['year'].value_counts().sort_index()

year_occurrences

In [ ]:
grouped = filtered_df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])

# Count all instances in each duplicate group
duplicate_counts = grouped.size()

# Calculate the total number of rows to drop (all duplicates minus one per group)
rows_to_drop = duplicate_counts.sum() - len(duplicate_counts)

rows_to_drop

In [8]:
print(filtered_df.duplicated().sum())
filtered_df = filtered_df.drop_duplicates()
filtered_df.info()


25497
<class 'pandas.core.frame.DataFrame'>
Index: 555240 entries, 5199 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  555231 non-null  object        
 1   resource_id                    555240 non-null  object        
 2   resource_type                  555240 non-null  object        
 3   time_call_received             555240 non-null  datetime64[ns]
 4   time_call_processed            555240 non-null  datetime64[ns]
 5   time_ambulance_notified        554939 non-null  datetime64[ns]
 6   time_dispatch                  539467 non-null  datetime64[ns]
 7   time_arrival_scene             493393 non-null  datetime64[ns]
 8   time_departure_scene           416375 non-null  datetime64[ns]
 9   time_arrival_hospital          414308 non-null  datetime64[ns]
 10  time_available                 555217 non-null  datetime64[ns]
 

In [ ]:
grouped = df.groupby(['time_call_received', 'x_accurate', 'y_accurate'])

# Count all instances in each duplicate group
duplicate_counts = grouped.size()

# Calculate the total number of rows to drop (all duplicates minus one per group)
rows_to_drop = duplicate_counts.sum() - len(duplicate_counts)

rows_to_drop

In [ ]:
# Group by day and count incidents
daily_incidents = df.groupby(df['time_call_received'].dt.date).size()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(daily_incidents.index, daily_incidents.values)

# Formatting x-axis to show only the year
plt.gca().xaxis.set_major_locator(mdates.YearLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

# Labels and title
plt.xlabel('Year')
plt.ylabel('Total Incidents')
plt.title('Total Incidents Per Day Over Years')

plt.grid(True)
plt.show()

In [5]:
dataframe = data_loader.processed_incidents_df.copy()

dataframe = data_preprocessor._remove_incomplete_years(dataframe)
dataframe = data_preprocessor._remove_duplicates(dataframe)

dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 555240 entries, 5199 to 754810
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   triage_impression_during_call  555231 non-null  object        
 1   resource_id                    555240 non-null  object        
 2   resource_type                  555240 non-null  object        
 3   time_call_received             555240 non-null  datetime64[ns]
 4   time_call_processed            555240 non-null  datetime64[ns]
 5   time_ambulance_notified        554939 non-null  datetime64[ns]
 6   time_dispatch                  539467 non-null  datetime64[ns]
 7   time_arrival_scene             493393 non-null  datetime64[ns]
 8   time_departure_scene           416375 non-null  datetime64[ns]
 9   time_arrival_hospital          414308 non-null  datetime64[ns]
 10  time_available                 555217 non-null  datetime64[ns]
 11  gr

In [6]:
dataframe.duplicated(subset=['time_call_received', 'x_accurate', 'y_accurate'], keep=False)

5199       True
5203       True
5205       True
5206      False
5207       True
          ...  
754806    False
754807    False
754808    False
754809     True
754810    False
Length: 555240, dtype: bool

In [7]:
datetime_columns = [
    "time_call_received", "time_call_processed", "time_ambulance_notified",
    "time_dispatch", "time_arrival_scene", "time_departure_scene",
    "time_arrival_hospital", "time_available"
]

# Count rows that violate the sequence
violations = 0
for i in range(len(datetime_columns) - 1):
    first_col = datetime_columns[i]
    second_col = datetime_columns[i + 1]
    
    # Count where the first date is after the second date
    violations += dataframe[dataframe[first_col] > dataframe[second_col]].shape[0]

violations


59901

In [8]:
dataframe["resource_type"].value_counts()

resource_type
Ambulanse         535332
Operativ Leder     13328
Legebil             5146
Akuttbil            1298
Syketransport        136
Name: count, dtype: int64